In [ ]:
from fastai_local.test import *

## Utility functions

In [ ]:
# export
def ifnone(a, b):
    "`b` if `a` is None else `a`"
    return b if a is None else a

In [ ]:
show_doc(ifnone)

<h4 id="ifnone" class="doc_header"><code>ifnone</code><a class="source_link" data-toggle="collapse" data-target="#ifnone-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>ifnone</code>(**`a`**, **`b`**)

<div class="collapse" id="ifnone-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#ifnone-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>ifnone</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

`b` if `a` is None else `a`  

Since `b if a is None else a` is such a common pattern, we wrap it in a function. However, be careful, because python will evaluate *both* `a` and `b` when calling `ifnone` (which it doesn't do if using the `if` version directly).

In [ ]:
test_eq(ifnone(None,1), 1)
test_eq(ifnone(2   ,1), 2)

In [ ]:
# export
def noop (      x, *args, **kwargs):
    "Do nothing - function"
    return x

def noops(self, x, *args, **kwargs):
    "Do nothing - method"
    return x

In [ ]:
show_doc(noop)

<h4 id="noop" class="doc_header"><code>noop</code><a class="source_link" data-toggle="collapse" data-target="#noop-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>noop</code>(**`x`**, **\*`args`**, **\*\*`kwargs`**)

<div class="collapse" id="noop-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#noop-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>noop</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Do nothing - function  

In [ ]:
test_eq(noop(1),1)

In [ ]:
show_doc(noops)

<h4 id="noops" class="doc_header"><code>noops</code><a class="source_link" data-toggle="collapse" data-target="#noops-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>noops</code>(**`x`**, **\*`args`**, **\*\*`kwargs`**)

<div class="collapse" id="noops-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#noops-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>noops</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Do nothing - method  

In [ ]:
class _t(): foo=noops
test_eq(_t().foo(1),1)

In [ ]:
def range_of(x): return list(range(len(x)))

In [ ]:
torch.Tensor.ndim = property(lambda x: x.dim())

In [ ]:
def tensor(x, *rest):
    "Like `torch.as_tensor`, but handle lists too, and can pass multiple vector elements directly."
    if len(rest): x = tuplify(x)+rest
    # Pytorch bug in dataloader using num_workers>0
    if is_listy(x) and len(x)==0: return tensor(0)
    res = torch.tensor(x) if is_listy(x) else as_tensor(x)
    if res.dtype is torch.int32:
        warn('Tensor is int32: upgrading to int64; for better performance use int64 input')
        return res.long()
    return res

In [ ]:
# export
#Rank 0 tensors in PyTorch are not really iterable
def is_iter(o): return isinstance(o, (Iterable,Generator)) and getattr(o,'ndim',1)

def listify(o):
    "Make `o` a list."
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, (str,np.ndarray,Tensor)): return [o]
    if is_iter(o): return list(o)
    return [o]

def tuplify(o): return tuple(listify(o))

In [ ]:
# test
test_eq(listify(None),[])
test_eq(listify([1,2,3]),[1,2,3])
test_ne(listify([1,2,3]),[1,2,])
test_eq(listify('abc'),['abc'])
test_eq(listify(range(0,3)),[0,1,2])
test_eq(listify(o for o in range(0,3)),[0,1,2])
test_eq(listify(tensor(0)),[tensor(0)])
test_eq(listify([tensor(0),tensor(1)]),[tensor(0),tensor(1)])
test_equal(listify(tensor([0.,1.1]))[0],tensor([0.,1.1]))

test_eq(tuplify(None),())
test_eq(tuplify([1,2,3]),(1,2,3))

In [ ]:
# export
def compose(*funcs): return reduce(lambda f,g: lambda x: f(g(x)), reversed(funcs), noop)
def is_listy(x:Any)->bool: return isinstance(x, (tuple,list))

In [ ]:
# export
def mask2idxs(mask): return [i for i,m in enumerate(mask) if m]

def uniqueify(x, sort=False, bidir=False, start=None):
    "Return the unique elements in `x`, optionally `sort`-ed, optionally return the reverse correspondance."
    res = list(OrderedDict.fromkeys(x).keys())
    if start is not None: res = listify(start)+res
    if sort: res.sort()
    if bidir: return res, {v:k for k,v in enumerate(res)}
    return res

In [ ]:
# test
test_eq(set(uniqueify([1,1,0,5,0,3])),{0,1,3,5})
test_eq(uniqueify([1,1,0,5,0,3], sort=True),[0,1,3,5])
v,o = uniqueify([1,1,0,5,0,3], bidir=True)
test_eq(v,[1,0,5,3])
test_eq(o,{1:0, 0: 1, 5: 2, 3: 3})
v,o = uniqueify([1,1,0,5,0,3], sort=True, bidir=True)
test_eq(v,[0,1,3,5])
test_eq(o,{0:0, 1: 1, 3: 2, 5: 3})

In [ ]:
# export
def setify(o): return o if isinstance(o,set) else set(listify(o))

In [ ]:
# test
test_eq(setify(None),set())
test_eq(setify('abc'),{'abc'})
test_eq(setify([1,2,2]),{1,2})
test_eq(setify(range(0,3)),{0,1,2})
test_eq(setify({1,2}),{1,2})